# Геокодирование на примере Yandex API


Простыми словами, **геокодирование** – это процесс, когда мы берём адрес или название места и превращаем его в точку на карте.
(Если делать наоборот – получать адрес по координатам, это называется **обратное геокодирование**.)

Чаще всего геокодирование делается с помощью специального сервиса, к которому мы обращаемся через **API**.

🔗 **API** (application programming interface) – это набор инструкций и стандартов, которые позволяют разным программам взаимодействовать друг с другом.
Проще говоря, **API – это посредник**, который даёт возможность использовать функционал другой программы.

--

Процесс геокодирования можно свести к нескольким шагам:

1. Пишем запрос к сервису через API (указываем адрес и другие важные параметры)
2. Получаем ответ
3. Обрабатываем ответ

--

На **Python** мы отправляем такие запросы с помощью библиотеки **requests**.
Если запрос выполнен успешно, мы получаем ответ и дальше работаем с ним: например, вытаскиваем координаты.

Куда именно отправлять запрос и какие параметры указывать – всегда написано в **документации к API** (их много разных; часть можно посмотреть [здесь](https://bella-mir.github.io/openSpData/)).

--

Здесь мы рассмотрим процесс геокодирования на примере **Yandex API**. Перед работой важно ознакомиться с [условиями](https://yandex.ru/dev/commercial/doc/ru/?from=mapsapi) бесплатного использования


## 0. Импортируем библиотеки


In [ ]:
# Для работ с табличными и пространственными данными

import pandas as pd
import geopandas as gpd

# Для запросов к API
import requests
import time  # для задержки между запросами, чтобы не заблокировали

## 1. Основы геокодирования


Мы рассмотрим основные шаги геокодирования на примере **Yandex API** для одного адреса:
определим параметры запроса, отправим его, разберём, из каких частей состоит ответ, и научимся извлекать из него координаты.

Всю информацию о работе с **Yandex API** можно (важно) посмотреть в [документации](https://yandex.ru/maps-api/docs/geocoder-api/index.html)


### 1.1 Определение параметров для запроса


Простыми словами запрос – это сообщение, которое мы отправляем какому-то серверу, чтобы получить или передать информацию.

Когда мы отправляем запрос, нам нужно объяснить сервису, что именно мы хотим. Для этого мы передаём параметры запроса.

В нашем случае параметры будут следующими:

- сам **адрес**, который нужно найти
- **API-ключ**, чтобы наш запрос был рассмаотрен
- в каком **формате** мы хотим получить ответ (JSON или XML)
- сколько результатов нам нужно (1 или несколько)
- на каком **языке** хотим получить ответ


In [ ]:
API_KEY = 'api_key'  # Ваш API-ключ Яндекс Геокодера

params = {
    'apikey': API_KEY, # Ключ доступа к API
    'geocode': 'Москва, Волхонка, д.12', # Адрес для геокодирования
    'format': 'json', # Формат ответа
    'results': 1, # Ограничить количество результатов до 1
    'lang': 'ru_RU', # Язык ответа
}

# URL для запроса к Яндекс Геокодеру
url = 'https://geocode-maps.yandex.ru/1.x/'



### 1.2 Отправка запроса


GET-запрос чаще всего используется, когда мы хотим получить данные (какой именно конкретный тип запроса нужен важно смотреть в [документации](https://yandex.ru/maps-api/docs/geocoder-api/index.html))

С помощью библиотеки requests мы отправляем запрос, в котором указываем url и параметры.

В ответ мы получаем код статуса - он показывает, как обработан наш запрос.

Основные статусы

- **200 OK** – всё прошло успешно, данные получены.
- **400 Bad Request** – неправильный запрос (например, ошибка в параметрах).
- **401 Unauthorized** – нет доступа, нужен правильный API-ключ или токен.
- **403 Forbidden** – доступ запрещён, даже если ключ есть.
- **404 Not Found** – адрес или ресурс не найден.
- **500 Internal Server Error** – ошибка на стороне сервера.


In [ ]:
# Отправляем
response = requests.get(url, params=params)

# Проверяем ответ
if response.status_code == 200:
    data = response.json()
    print("Успешный запрос")
else:
    print("Ошибка:", response.status_code)



### 1.3 Извлечение ответа


Если запрос прошел успешно, то мы смотрим, что у нас получилось в ответе, и извлекаем из него нужную информацию


In [ ]:
data

Давайте создадим DataFrame, в котором будут сохранены адрес, почтовый индекс и координаты


In [ ]:
# Достаем блок GeoObject
geo = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']

# Разбиваем координаты на долготу и широту
pos = geo['Point']['pos'].split()
lon = float(pos[0])
lat = float(pos[1])

# Достаем адрес
address_meta = geo['metaDataProperty']['GeocoderMetaData']['Address']
components = address_meta['Components']

# Создаем словарь с нужными полями
result = {
    'formatted_address': address_meta['formatted'],
    'postal_code': address_meta.get('postal_code', None),
    'longitude': lon,
    'latitude': lat,
}


# Создаем DataFrame
df = pd.DataFrame([result])


# Смотрим на результат
df

### 1.4 Создаем набор пространственных данных


На основе полученного dataFrame мы можем создать geoDataFrame, используя координаты объекта


In [ ]:
#Создаем geoDataFrame
data_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs=4326)

data_gdf.head()

## 2. Геокодирование данных из таблицы


У нас довольно редко возникает задача геокодирования **только одного конкретного адреса**. Намного чаще у нас есть **набор адресов** в таблице, которые нужно геокодировать.

Давайте посмотрим на небольшом примере (**df_geocode_sample**), как это можно сделать.


Изучим имеющийся набор данных


In [ ]:
data_sample = pd.read_csv('./df_geocode_sample.csv', sep=';', on_bad_lines='skip')

Наша задача остаётся той же, но теперь нам нужно обработать все адреса: отправить для каждого запрос, получить ответ и сохранить координаты.


### 2.1 Проходим по каждому адресу из таблицы


In [ ]:
# Создаем пустой список для результатов
results = []

# URL для Яндекс Геокодера
url = 'https://geocode-maps.yandex.ru/1.x/'

# Общие параметры
params = {
    'apikey': API_KEY, # Ключ доступа к API
    'format': 'json', # Формат ответа
    'results': 1, # Ограничить количество результатов до 1
    'lang': 'ru_RU', # Язык ответа
}

# Проходим по каждой строке DataFrame
for idx, row in data_sample.head(10).iterrows():
    address = row['Address']

    # Создаём копию общих параметров и добавляем адрес
    params_request = params.copy()
    params_request['geocode'] = address

    response = requests.get(url, params=params_request)
    print(f"Processing id {row['id']} - status {response.status_code}")

    if response.status_code == 200:
        data = response.json()

        # Достаем блок GeoObject
        geo = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']
        
        # Разбиваем координаты на долготу и широту
        pos = geo['Point']['pos'].split()
        lon = float(pos[0])
        lat = float(pos[1])

        # Достаем адрес
        address_meta = geo['metaDataProperty']['GeocoderMetaData']['Address']
        components = address_meta['Components']

        # Создаем словарь с нужными полями
        result = {
            'id': row['id'],
            'name': row['CommonName'],
            'input_address': address,
            'formatted_address': address_meta['formatted'],
            'postal_code': address_meta.get('postal_code', None),
            'longitude': lon,
            'latitude': lat,
            }
        
        # Добавлем результат в список
        results.append(result)
        
    else:
        print(f"Ошибка запроса для id {row['id']}")
        results.append({
            'id': row['id'],
            'name': row['CommonName'],
            'input_address': address,
            'formatted_address': None,
            'postal_code': None,
            'longitude': None,
            'latitude': None,
        })

    time.sleep(0.2)  # Задержка, чтобы избежать превышения лимитов API

# Создаем финальный DataFrame
final_df = pd.DataFrame(results)




### 2.2 Создаем GeoDataFrame на основе полученных координат


In [ ]:
#Убираем строки с пустыми координатами
final_df_cleaned = final_df.dropna(subset=['longitude', 'latitude'])

#Создаем geoDataFrame
final_gdf = gpd.GeoDataFrame(final_df_cleaned, geometry=gpd.points_from_xy(final_df_cleaned['longitude'], final_df_cleaned['latitude']), crs=4326)

#Смотрим на результат
final_gdf.head()

## 3. Функция геокодирования


Геокодирование – довольно частая задача, и каждый раз записывать все шаги и параметры не очень удобно. Поэтому давайте на основе нашего кода выше напишем **небольшую функцию для геокодирования**, которую можно будет использовать в дальнейшем.


In [ ]:
def geocode_addresses(api_key, df, address_column):
    """
    Геокодирование адресов через Yandex API с выводом GeoDataFrame
    
    :param api_key: str, API-ключ Яндекса
    :param df: pandas.DataFrame, таблица с адресами
    :param address_column: str, название столбца с адресами
    :return: geopandas.GeoDataFrame, таблица с координатами и геометрией
    """
    
    results = []
    
    url = 'https://geocode-maps.yandex.ru/1.x/'
    
    # Общие параметры (без адреса)
    base_params = {
        'apikey': api_key,
        'format': 'json',
        'results': 1,
        'lang': 'ru_RU',
    }
    
    for idx, row in df.iterrows():
        address = row[address_column]
        
        params_request = base_params.copy()
        params_request['geocode'] = address
        
        response = requests.get(url, params=params_request)
        print(f"Processing id {row.get('id', idx)} - status {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            try:
                geo = data['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']
                
                pos = geo['Point']['pos'].split()
                lon = float(pos[0])
                lat = float(pos[1])
                
                address_meta = geo['metaDataProperty']['GeocoderMetaData']['Address']
                
                result = {
                    'id': row.get('id', idx),
                    'input_address': address,
                    'formatted_address': address_meta.get('formatted', None),
                    'postal_code': address_meta.get('postal_code', None),
                    'longitude': lon,
                    'latitude': lat,
                }
                
            except IndexError:
                print(f"Адрес не найден: {address}")
                result = {
                    'id': row.get('id', idx),
                    'input_address': address,
                    'formatted_address': None,
                    'postal_code': None,
                    'longitude': None,
                    'latitude': None,
                }
                
        else:
            print(f"Ошибка запроса для id {row.get('id', idx)}")
            result = {
                'id': row.get('id', idx),
                'input_address': address,
                'formatted_address': None,
                'postal_code': None,
                'longitude': None,
                'latitude': None,
            }
        
        results.append(result)
        time.sleep(0.2)  # задержка для предотвращения блокировки API
    
    # Создаём финальный DataFrame
    final_df = pd.DataFrame(results)
    
    # Убираем строки с пустыми координатами
    final_df_cleaned = final_df.dropna(subset=['longitude', 'latitude'])
    
    # Создаём GeoDataFrame
    final_gdf = gpd.GeoDataFrame(
        final_df_cleaned,
        geometry=gpd.points_from_xy(final_df_cleaned['longitude'], final_df_cleaned['latitude']),
        crs='EPSG:4326'
    )
    
    return final_gdf

Проверим, как работает наша функция на основе нескольких адресаов


In [ ]:
# Запусаем функцию
goeocoded = geocode_addresses(API_KEY, data_sample, 'Address')

#Смотрим на результат
goeocoded.head()

## 4. Итог


На примере **Yandex API** мы научились геокодировать адреса из таблиц, но самое главное – в целом разобрались, как устроен процесс геокодирования.
Вы можете использовать такой подход для любых API, изменяя параметры запроса и обработку ответа в соответствии с их документацией.
Также **не забывайте проверять лимиты** на количество бесплатных запросов.

Успехов!
